<a href="https://colab.research.google.com/github/RyanBadai/Job-Scraping/blob/main/job_scraping.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import pandas as pd
import requests
from bs4 import BeautifulSoup

In [4]:
url_jobstreet = f'https://id.jobstreet.com/id/Data-Analyst-jobs?page={1}&sortmode=ListedDate'
s = requests.session()
page = s.get(url_jobstreet)
soup = BeautifulSoup(page.text, "lxml")
joblist = soup.find('div', class_='gepq850 eihuid5h eihuid53')

print(joblist)

<div class="gepq850 eihuid5h eihuid53"><a class="gepq850 gepq85f gepq850 gepq85f k5mille k5millg" data-automation="jobTitle" data-run-click-only="true" data-testid="job-card-title" href="/id/job/82986249?type=standard&amp;ref=search-standalone&amp;origin=cardTitle" id="job-title-82986249" rel="nofollow" target="_self">Business Analyst</a></div>


In [5]:
req = requests.get(url_jobstreet)
req

<Response [200]>

In [9]:
import requests
from bs4 import BeautifulSoup
import pandas as pd
import signal

# Fungsi timeout handler
class TimeoutException(Exception):
    pass

def timeout_handler(signum, frame):
    raise TimeoutException

# Fungsi untuk scraping halaman utama
def scrape_main_page():
    jobs_data = []
    for i in range(1, 5):  # Scrape halaman 1-4
        url = f'https://id.jobstreet.com/id/Digital-Marketing-jobs?page={i}&sortmode=ListedDate'
        headers = {
            "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/129.0.6668.90"
        }
        s = requests.session()
        s.headers.update(headers)
        try:
            page = s.get(url, timeout=60)  # Tambahkan timeout di request
            soup = BeautifulSoup(page.text, "lxml")
            joblist = soup.find('div', class_='gepq850 _21bfxf1')
            if joblist is None:
                print(f"Job list not found on page {i}. Check the selector.")
                continue  # Skip ke halaman berikutnya
            for artikel in joblist.find_all('article', {'class': 'gepq850'}):
                posisi = artikel.find('div', {'class': 'gepq850 eihuid5h eihuid53'})
                perusahaan = artikel.find(attrs={'data-automation': 'jobCompany'})
                lokasi = artikel.find(attrs={'data-automation': 'jobLocation'})
                date = artikel.find(attrs={'data-automation': 'jobListingDate'})
                job_url = artikel.find('a', href=True)
                posisi_text = posisi.get_text() if posisi else "Posisi tidak ditemukan"
                perusahaan_text = perusahaan.get_text() if perusahaan else "Perusahaan tidak disebutkan"
                lokasi_text = lokasi.get_text() if lokasi else "Lokasi tidak ditemukan"
                date_text = date.get_text() if date else "Tanggal tidak ditemukan"
                job_url_text = 'https://id.jobstreet.com' + job_url['href'] if job_url and job_url['href'].startswith('/') else job_url['href'] if job_url else "URL tidak ditemukan"

                jobs_data.append({
                    "Posisi": posisi_text,
                    "Perusahaan": perusahaan_text,
                    "Lokasi": lokasi_text,
                    "Tanggal": date_text,
                    "URL": job_url_text
                })
        except requests.exceptions.RequestException as e:
            print(f"Error: {e}")
    return jobs_data

# Fungsi untuk scraping halaman detail dengan timeout
def scrape_detail_page(url):
    headers = {
        "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/129.0.6668.90"
    }

    # Atur timeout untuk fungsi ini (Unix-based OS)
    signal.signal(signal.SIGALRM, timeout_handler)
    signal.alarm(60)  # Timeout dalam 60 detik

    try:
        page = requests.get(url, headers=headers, timeout=60)
        soup = BeautifulSoup(page.text, "lxml")

        # Ekstrak deskripsi pekerjaan
        description_div = soup.find('div', {'data-automation': 'jobAdDetails'})
        description_text = description_div.get_text(strip=True) if description_div else "Deskripsi tidak ditemukan"

        signal.alarm(0)  # Matikan alarm jika sukses
        return description_text

    except TimeoutException:
        print(f"Timeout saat scraping: {url}")
        return "Timeout - Deskripsi tidak ditemukan"

    except requests.exceptions.RequestException as e:
        print(f"Error scraping detail page {url}: {e}")
        return "Deskripsi tidak ditemukan"

    finally:
        signal.alarm(0)  # Pastikan alarm dimatikan

# Main program
if __name__ == "__main__":
    # Step 1: Scrape halaman utama
    jobs_data = scrape_main_page()

    # Step 2: Scrape halaman detail untuk setiap URL
    for job in jobs_data:
        print(f"Scraping detail for: {job['Posisi']} ({job['URL']})")
        job['Deskripsi'] = scrape_detail_page(job['URL'])

    # Step 3: Simpan hasil ke CSV
    df = pd.DataFrame(jobs_data)
    df.to_csv("detailed_jobs_data.csv", index=False, encoding="utf-8")
    print("Data saved to detailed_jobs_data.csv")

Scraping detail for: Sales Take Order /  Marketing Alat-alat Teknik (https://id.jobstreet.com/id/job/82990299?type=standard&ref=search-standalone)
Scraping detail for: Social Media Specialist (https://id.jobstreet.com/id/job/82989940?type=standard&ref=search-standalone)
Scraping detail for: Digital Marketing (https://id.jobstreet.com/id/job/82989745?type=standard&ref=search-standalone)
Scraping detail for: Marketing Communication (https://id.jobstreet.com/id/job/82935566?type=standard&ref=search-standalone)
Scraping detail for: Online Sales & Marketing (https://id.jobstreet.com/id/job/82989895?type=standard&ref=search-standalone)
Scraping detail for: Telemarketing - Surakarta Site (https://id.jobstreet.com/id/job/82603330?type=standard&ref=search-standalone)
Scraping detail for: Marketing Officer (Medan) (https://id.jobstreet.com/id/job/82988966?type=standard&ref=search-standalone)
Scraping detail for: Marketing Officer (Pekanbaru) (https://id.jobstreet.com/id/job/82988961?type=standar

In [10]:
df.head()

,Posisi,Perusahaan,Lokasi,Tanggal,URL,Deskripsi
0,Sales Take Order / Marketing Alat-alat Teknik,PT Jaya Pratama Groserindo,Denpasar,1 jam yang lalu,https://id.jobstreet.com/id/job/82990299?type=...,Penjualan Produk:Memasarkan dan menjual alat-a...
1,Social Media Specialist,PT TRILOGI BERJAYA BERSAMA,Tangerang,1 jam yang lalu,https://id.jobstreet.com/id/job/82989940?type=...,KUALIFIKASI :- Minimal Lulusan D3/S1(Diutamaka...
2,Digital Marketing,PT EKIS MUDA BERKARYA,Surabaya,2 jam yang lalu,https://id.jobstreet.com/id/job/82989745?type=...,KeahlianMenguasai berbagai platform media sosi...
3,Marketing Communication,PT EKIS MUDA BERKARYA,Surabaya,2 jam yang lalu,https://id.jobstreet.com/id/job/82935566?type=...,Job Desc:Mengembangkan dan menerapkan strategi...
4,Online Sales & Marketing,RNB Meatshop - Yasmin,Cibadak,5 jam yang lalu,https://id.jobstreet.com/id/job/82989895?type=...,Kualifikasi pekerjaan:Tersedia jam fleksibelDi...


In [8]:
jobs_data = []

for i in range(1,5):
  url = f'https://id.jobstreet.com/id/Data-Analyst-jobs?page={i}&sortmode=ListedDate'

  headers = {
      "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/129.0.6668.90"
    }

  s = requests.session()
  s.headers.update(headers)

  try:
      page = s.get(url)
      soup = BeautifulSoup(page.text, "lxml")
      joblist = soup.find('div', class_='gepq850 _21bfxf1')

      if joblist is None:
            print(f"Job list not found on page {i}. Check the selector.")
            continue # Skip to the next page

      for artikel in joblist.find_all('article', {'class': 'gepq850'}):
          posisi = artikel.find('div', {'class' : 'gepq850 eihuid5h eihuid53'})
          perusahaan = artikel.find(attrs={'data-automation':'jobCompany'})
          lokasi = artikel.find(attrs={'data-automation':'jobLocation'})
          gaji = artikel.find('span', {'class':'_1fggenz0 _1feq2e74z _1feq2e7r _474bdu0 _474bdu1 _474bdu21 v7shb4 _474bdu7 _9ci8fv0'})
          waktu = artikel.find('span',{'class':'_1fggenz0 _1feq2e74z _474bdu0 _474bdu1 _474bdu22 v7shb4 _474bdu7'})
          job_url = artikel.find('a', href=True)

          posisi_text = posisi.get_text() if posisi else "Posisi tidak ditemukan"
          perusahaan_text = perusahaan.get_text() if perusahaan else "Perusahaan tidak disebutkan"
          lokasi_text = lokasi.get_text() if lokasi else "Lokasi tidak ditemukan"
          gaji_text = gaji.get_text() if gaji else "Gaji tidak disebutkan"
          waktu_text = waktu.get_text() if waktu else "Waktu tidak ditemukan"
          job_url_text = 'https://id.jobstreet.com' + job_url['href'] if job_url and job_url['href'].startswith('/') else job_url['href'] if job_url else "URL tidak ditemukan"

          jobs_data.append({
            "Posisi": posisi_text,
            "Perusahaan": perusahaan_text,
            "Lokasi": lokasi_text,
            "Gaji": gaji_text,
            "Waktu": waktu_text,
            "URL": job_url_text
          })

          print(posisi_text)
          print(job_url_text)
          # print(perusahaan_text)
          # print(lokasi_text)
          # print(gaji_text)
          # print(waktu_text)
          print("===============================")
  except requests.exceptions.RequestException as e:
      print(f"Error: {e}")

Business Analyst
https://id.jobstreet.com/id/job/82986249?type=standard&ref=search-standalone
PA Marketplace & Ads Specialist
https://id.jobstreet.com/id/job/82982127?type=standard&ref=search-standalone
Data Annotator - Transsion Indonesia
https://id.jobstreet.com/id/job/82980115?type=standard&ref=search-standalone
Finance Analyst
https://id.jobstreet.com/id/job/82979409?type=standard&ref=search-standalone
SEO
https://id.jobstreet.com/id/job/82973308?type=standard&ref=search-standalone
Data & Analytics Engineer
https://id.jobstreet.com/id/job/82974109?type=standard&ref=search-standalone
Lark Development Specialist
https://id.jobstreet.com/id/job/82970747?type=standard&ref=search-standalone
Data Engineer
https://id.jobstreet.com/id/job/82971848?type=standard&ref=search-standalone
Specialist, Business Analyst (Paper)
https://id.jobstreet.com/id/job/82969141?type=standard&ref=search-standalone
IT Business Analyst Staff
https://id.jobstreet.com/id/job/82968549?type=standard&ref=search-stan

In [ ]:
df_jobs_full = pd.DataFrame(jobs_data)

df_jobs_full.head()

,Posisi,Perusahaan,Lokasi,Gaji,Waktu,URL
0,Data Scientist,PT IDStar Cipta Teknologi,Jakarta Selatan,Gaji tidak disebutkan,Waktu tidak ditemukan,https://id.jobstreet.com/id/job/82136640?type=...
1,Claim Analyst,Pengiklan Anonim,Jakarta Selatan,Gaji tidak disebutkan,Waktu tidak ditemukan,https://id.jobstreet.com/id/job/82136443?type=...
2,Sales Analyst & Operation Support,PT Eka Mas Republik,Jakarta Raya,Gaji tidak disebutkan,Waktu tidak ditemukan,https://id.jobstreet.com/id/job/82136066?type=...
3,Business Specialist,ENO INTERNATIONAL TRADING INC.,Jakarta Raya,Gaji tidak disebutkan,Waktu tidak ditemukan,https://id.jobstreet.com/id/job/82135994?type=...
4,Pricing Analyst,PT KEX NEX EXPRESS,Jakarta Timur,Gaji tidak disebutkan,Waktu tidak ditemukan,https://id.jobstreet.com/id/job/82135044?type=...


In [ ]:
df_jobs_full.shape

(128, 6)

# Glints

In [ ]:
url = f'https://glints.com/vn/en/opportunities/jobs/explore?keyword=digital+marketing&country=ID&locationName=All+Cities%2FProvinces&page=2'
s = requests.session()
s.headers.update(headers)
page = s.get(url)
soup = BeautifulSoup(page.text, "lxml")
joblist = soup.find('div', class_='JobCardsc__JobcardContainer-sc-hmqj50-0 iirqVR CompactOpportunityCardsc__CompactJobCardWrapper-sc-dkg8my-5 hRilQl')

print(joblist)

None


In [ ]:
jobs_data = []

for i in range(1,5):
  url = f'https://glints.com/vn/en/opportunities/jobs/explore?keyword=digital+marketing&country=ID&locationName=All+Cities%2FProvinces&page={i}'

  headers = {
      "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/129.0.6668.90"
    }

  s = requests.session()
  s.headers.update(headers)

  try:
      page = s.get(url)
      soup = BeautifulSoup(page.text, "lxml")
      joblist = soup.find('div', class_='MainContainersc__MainBody-sc-xmyke8-2 drnzBQ')

      if joblist is None:
            print(f"Job list not found on page {i}. Check the selector.")
            continue # Skip to the next page

      for artikel in joblist.select('div', {'class': 'JobCardsc__JobCardWrapper-sc-hmqj50-1 fnsbDp'}):
          posisi = artikel.find('div', {'class' : 'CompactOpportunityCardsc__JobTitle-sc-dkg8my-11 gwOhhF'})
          link = artikel.find('div', {'class' : 'CompactOpportunityCardsc__CompactJobCard-sc-dkg8my-4 ipGhBs job-search-results_job-card_link'})

          posisi_text = posisi.get_text() if posisi else "Posisi tidak ditemukan"
          link_text = link.get_text() if posisi else "Posisi tidak ditemukan"

          jobs_data.append({
            "Posisi": posisi_text,
            "Link": link_text
          })

          print(posisi_text)
          print(link_text)
          # print(perusahaan_text)
          # print(lokasi_text)
          # print(gaji_text)
          # print(waktu_text)
          print("===============================")
  except requests.exceptions.RequestException as e:
      print(f"Error: {e}")

Job list not found on page 1. Check the selector.
Job list not found on page 2. Check the selector.
Job list not found on page 3. Check the selector.
Job list not found on page 4. Check the selector.


# Testing

In [ ]:
import requests
from bs4 import BeautifulSoup
import csv

In [ ]:
url_glints = "https://glints.com/vn/en/opportunities/jobs/explore?keyword=digital+marketing&country=ID&locationName=All+Cities%2FProvinces"

In [ ]:
headers = {
      "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/129.0.6668.90"
    }

s = requests.session()
s.headers.update(headers)

In [ ]:
req = s.get(url_glints)
req

<Response [403]>

In [ ]:
soup = BeautifulSoup(req.text, "html.parser")

In [ ]:
semua = soup.find_all('div', class_='ExploreTabsc__Body-sc-1fr7yeh-3 bFDfCF')
print(semua)

[]
